# API du parlement

http://ws-old.parlament.ch/

«Pour le moment, les nouveaux services web ne sont pas encore disponibles. Nous sommes cependant certains que nous pourrons remédier à cette situation d’ici au troisième trimestre 2019.»

Votes:
http://ws-old.parlament.ch/votes/councillors

### 1. Récupérer la liste actuelle des parlementaires

In [1]:
import pandas as pd
import requests

# Largeur de colonne illimitée
pd.set_option("display.max_colwidth", None)

In [2]:
# On récupère la liste des 246 parlementaires
response = requests.get(
    "http://ws-old.parlament.ch/councillors/basicdetails?format=json&lang=fr",
    headers={
        "Accept": "application/json",
        "Accept-Language": "fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5",
    },
)

In [3]:
df_councillors = pd.DataFrame(response.json())
len(df_councillors)

248

In [4]:
df_councillors.head()

,id,updated,biographyUrl,canton,cantonName,council,faction,factionName,firstName,function,homepage,lastName,number,party,partyName,pictureUrl
0,4154,2023-11-10T10:02:47Z,http://www.parlament.ch/e/suche/Pages/biografie.aspx?biografie_id=4154,VS,Valais,N,V,Groupe de l'Union démocratique du Centre,Jean-Luc,Membre,www.jladdor.ch,Addor,3055,UDC,Union Démocratique du Centre,http://www.parlament.ch/SiteCollectionImages/profil/225x225/3055.jpg
1,3867,2023-11-10T10:02:47Z,http://www.parlament.ch/e/suche/Pages/biografie.aspx?biografie_id=3867,BE,Berne,N,V,Groupe de l'Union démocratique du Centre,Andreas,Membre,www.aebi-andreas.ch,Aebi,2670,UDC,Union Démocratique du Centre,http://www.parlament.ch/SiteCollectionImages/profil/225x225/2670.jpg
2,4049,2023-11-10T10:02:47Z,http://www.parlament.ch/e/suche/Pages/biografie.aspx?biografie_id=4049,BE,Berne,N,S,Groupe socialiste,Matthias,Membre,www.matthiasaebischer.ch,Aebischer,2760,PSS,Parti socialiste suisse,http://www.parlament.ch/SiteCollectionImages/profil/225x225/2760.jpg
3,4053,2023-11-13T17:57:19Z,http://www.parlament.ch/e/suche/Pages/biografie.aspx?biografie_id=4053,ZG,Zoug,N,V,Groupe de l'Union démocratique du Centre,Thomas,Président(e),www.aeschi.com,Aeschi,2758,UDC,Union Démocratique du Centre,http://www.parlament.ch/SiteCollectionImages/profil/225x225/2758.jpg
4,4090,2023-11-10T10:02:47Z,http://www.parlament.ch/e/suche/Pages/biografie.aspx?biografie_id=4090,GE,Genève,N,V,Groupe de l'Union démocratique du Centre,Céline,Membre,NaN,Amaudruz,2796,UDC,Union Démocratique du Centre,http://www.parlament.ch/SiteCollectionImages/profil/225x225/2796.jpg


In [5]:
# La colonne "council" contient une lettre correspondant à la chambre
df_councillors["council"].value_counts()

council
N    203
S     45
Name: count, dtype: int64

In [6]:
df_national = df_councillors[df_councillors["council"] == "N"]
df_etats = df_councillors[df_councillors["council"] == "S"]

### 2. Consulter les votes

Attention, les parlementaires ont deux numéros:
- id
- number

C'est number qu'il faut utiliser pour obtenir les votes.

In [7]:
df_etats[["firstName", "lastName", "id", "number"]].head()

,firstName,lastName,id,number
12,Philippe,Bauer,4187,3059
22,Pirmin,Bischof,3871,2674
33,Thierry,Burkart,4189,3065
35,Andrea,Caroni,4075,2775
37,Isabelle,Chassot,4325,3220


In [8]:
# Pour consulter les votes d’un parlementaire

numero_parlementaire = 3059
response = requests.get(
    f"http://ws-old.parlament.ch/votes/councillors/{numero_parlementaire}?format=json&lang=fr",
    headers={
        "Accept": "application/json",
        "Accept-Language": "fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5",
    },
)
response

<Response [200]>

In [9]:
# Voici le contenu de la réponse
response.json()

{'id': 3059,
 'updated': '2023-05-25T13:34:51Z',
 'elanId': 801,
 'firstName': 'Philippe',
 'lastName': 'Bauer',
 'affairVotes': [{'id': 17496,
   'affairId': 20020453,
   'affairTitle': 'La transformation des bâtiments en zone agricole. Une compétence cantonale',
   'councillorVote': {'id': 3002210, 'decision': 'Yes'},
   'date': '2016-06-17T09:00:55Z',
   'divisionText': None,
   'meaningNo': 'Antrag Page (keine Abschreibung)',
   'meaningYes': 'Antrag der Kommission (Abschreibung der Initiative)',
   'registrationNumber': 13756,
   'submissionText': ''},
  {'id': 17983,
   'affairId': 20050464,
   'affairTitle': 'Dépistage du cancer',
   'councillorVote': {'id': 3077008, 'decision': 'Yes'},
   'date': '2016-09-30T08:15:43Z',
   'divisionText': None,
   'meaningNo': 'Antrag der Minderheit Heim (Fristverlängerung)',
   'meaningYes': 'Antrag der Mehrheit (Abschreibung)',
   'registrationNumber': 14243,
   'submissionText': ''},
  {'id': 17209,
   'affairId': 20070402,
   'affairTitle':

In [10]:
# On voit que le 50e vote a un attribut "hasMorePages" avec la valeur "True"...
response.json()["affairVotes"][-1]

{'id': 17818,
 'hasMorePages': True,
 'affairId': 20120057,
 'affairTitle': 'Développement de l’acquis de Schengen. Reprise du règlement portant création d’une agence pour des systèmes d‘information',
 'councillorVote': {'id': 3056410, 'decision': 'Yes'},
 'date': '2016-09-27T09:22:24Z',
 'divisionText': None,
 'meaningNo': 'Ablehnung der Vorlage',
 'meaningYes': 'Annahme der Vorlage',
 'registrationNumber': 14078,
 'submissionText': 'Arrêté fédéral portant approbation de l’échange de notes entre la Suisse et l’UE concernant la reprise du règlement (UE) no 1077/2011 portant création d’une agence européenne pour la gestion opérationnelle des systèmes d’information à grande échelle (Développement de l’acquis de Schengen et de l’acquis «Dublin/Eurodac»)'}

## Problème de pagination

L’API renvoie seulement 50 votes. Et pas forcément les derniers. Plusieurs solutions sont possibles:
* filtrer par date, par exemple `http://ws-old.parlament.ch/votes/councillors/3055?dateFromFilter=2023/06/01`
* filtrer par législature, pour la 51e (2019-2023): `http://ws-old.parlament.ch/votes/councillors/3055?legislativePeriodFilter=51`
* consulter toutes les pages disponibles, par exemple `http://ws-old.parlament.ch/votes/councillors/3055?pageNumber=2` pour la 2e page, `?pageNumber=3` pour la 3e, etc. L’API renvoie la valeur «VRAI» dans `hasMorePages` si des pages supplémentaires sont disponibles.

Dans l’exemple ci-dessous, on va se cantonner aux derniers mois de la 51e législature.

In [11]:
# Pour consulter les votes d’un parlementaire

numero_parlementaire = 3055
response = requests.get(
    f"http://ws-old.parlament.ch/votes/councillors/{numero_parlementaire}?dateFromFilter=2023/06/01",
    headers={
        "Accept": "application/json",
        "Accept-Language": "fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5",
    },
)
response

<Response [200]>

In [12]:
# On peut mettre les votes du candidat dans un tableau.
# On voit que ça contient en fait plein de détails sur les votes
votes_df = pd.DataFrame(response.json()["affairVotes"])
votes_df.head()

,id,affairId,affairTitle,councillorVote,date,divisionText,meaningNo,meaningYes,registrationNumber,submissionText,hasMorePages
0,31385,20030424,Actes d'ordre sexuel avec des enfants. Allongement de la peine prévue par l'article 187 CP,"{'id': 5081394, 'decision': 'No'}",2023-09-28T17:58:40Z,None,Antrag der Minderheit Tuena (Fristverlängerung),Antrag der Mehrheit (Abschreibung),27645,,NaN
1,31029,20090528,Financement moniste des prestations de soins,"{'id': 5023994, 'decision': 'Yes'}",2023-09-13T17:11:02Z,None,Antrag der Minderheit Sauter,Antrag der Mehrheit,27289,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
2,31030,20090528,Financement moniste des prestations de soins,"{'id': 5024194, 'decision': 'No'}",2023-09-13T17:12:21Z,None,Antrag der Minderheit II Crottaz,Antrag der Minderheit I Maillard,27290,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
3,31031,20090528,Financement moniste des prestations de soins,"{'id': 5024394, 'decision': 'Yes'}",2023-09-13T17:13:05Z,None,Antrag der Minderheit II Crottaz,Antrag der Mehrheit,27291,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
4,31032,20090528,Financement moniste des prestations de soins,"{'id': 5024594, 'decision': 'Yes'}",2023-09-13T17:13:47Z,None,Antrag der Minderheit I Gysi Barbara,Antrag der Mehrheit,27292,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN


In [13]:
# Autre possibilité: enregistrer uniquement les réponses du candidat
votes_seuls = pd.DataFrame([i["councillorVote"] for i in response.json()["affairVotes"]])
print(len(votes_seuls))
votes_seuls.head()

50


,id,decision
0,5081394,No
1,5023994,Yes
2,5024194,No
3,5024394,Yes
4,5024594,Yes


In [14]:
# Pour nos essais, on fixe une limite à 5 parlementaires du National.
limit = 5
votes_data = []
for i, row in df_national[:limit].iterrows():
    name = row["firstName"] + " " + row["lastName"]
    url = f"http://ws-old.parlament.ch/votes/councillors/{row['number']}?dateFromFilter=2023/06/01"
    print(name, url)
    response = requests.get(
        url,
        headers={
            "Accept": "application/json",
            "Accept-Language": "fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5",
        },
    )
    print(len(response.json()["affairVotes"]), "votes trouvés.")
    votes_parlementaire = []
    for vote in response.json()["affairVotes"]:
        votes_parlementaire.append(
            {
                "affairId": vote["id"],
                name: vote["councillorVote"]["decision"],
            }
        )
    votes_data.append(votes_parlementaire)

Jean-Luc Addor http://ws-old.parlament.ch/votes/councillors/3055?dateFromFilter=2023/06/01
50 votes trouvés.
Andreas Aebi http://ws-old.parlament.ch/votes/councillors/2670?dateFromFilter=2023/06/01
50 votes trouvés.
Matthias Aebischer http://ws-old.parlament.ch/votes/councillors/2760?dateFromFilter=2023/06/01
50 votes trouvés.
Thomas Aeschi http://ws-old.parlament.ch/votes/councillors/2758?dateFromFilter=2023/06/01
50 votes trouvés.
Céline Amaudruz http://ws-old.parlament.ch/votes/councillors/2796?dateFromFilter=2023/06/01
50 votes trouvés.


### 3. Combiner les décisions pour chaque vote

In [15]:
df_list = []
for liste_votes in votes_data:
    dfv = pd.DataFrame(liste_votes)
    df_list.append(dfv.set_index("affairId"))

In [16]:
decisions_df = pd.concat(df_list, axis=1)
decisions_df.head()

,Jean-Luc Addor,Andreas Aebi,Matthias Aebischer,Thomas Aeschi,Céline Amaudruz
affairId,,,,,
31385,No,No,Yes,No,No
31029,Yes,Yes,No,Yes,Yes
31030,No,No,Yes,No,No
31031,Yes,Yes,No,Yes,Yes
31032,Yes,Yes,No,Yes,Yes


In [17]:
# On enlève la colonne avec les votes d'Addor du dataframe contenant
# les infos sur le vote, puisqu'on les a déjà dans un meilleur format
del votes_df["councillorVote"]

In [18]:
decisions_df.merge(votes_df, left_index=True, right_on="id")

,Jean-Luc Addor,Andreas Aebi,Matthias Aebischer,Thomas Aeschi,Céline Amaudruz,id,affairId,affairTitle,date,divisionText,meaningNo,meaningYes,registrationNumber,submissionText,hasMorePages
0,No,No,Yes,No,No,31385,20030424,Actes d'ordre sexuel avec des enfants. Allongement de la peine prévue par l'article 187 CP,2023-09-28T17:58:40Z,None,Antrag der Minderheit Tuena (Fristverlängerung),Antrag der Mehrheit (Abschreibung),27645,,NaN
1,Yes,Yes,No,Yes,Yes,31029,20090528,Financement moniste des prestations de soins,2023-09-13T17:11:02Z,None,Antrag der Minderheit Sauter,Antrag der Mehrheit,27289,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
2,No,No,Yes,No,No,31030,20090528,Financement moniste des prestations de soins,2023-09-13T17:12:21Z,None,Antrag der Minderheit II Crottaz,Antrag der Minderheit I Maillard,27290,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
3,Yes,Yes,No,Yes,Yes,31031,20090528,Financement moniste des prestations de soins,2023-09-13T17:13:05Z,None,Antrag der Minderheit II Crottaz,Antrag der Mehrheit,27291,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
4,Yes,Yes,No,Yes,Yes,31032,20090528,Financement moniste des prestations de soins,2023-09-13T17:13:47Z,None,Antrag der Minderheit I Gysi Barbara,Antrag der Mehrheit,27292,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
5,No,No,Yes,No,No,31033,20090528,Financement moniste des prestations de soins,2023-09-13T17:14:39Z,None,Antrag der Minderheit II Glarner,Antrag der Mehrheit,27293,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
6,Yes,Yes,No,Yes,Yes,31034,20090528,Financement moniste des prestations de soins,2023-09-13T17:15:16Z,None,Antrag der Minderheit III Weichelt,Antrag der Mehrheit,27294,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
7,No,No,Yes,No,No,31035,20090528,Financement moniste des prestations de soins,2023-09-13T17:15:53Z,None,Antrag der Minderheit Aeschi Thomas (streichen),Antrag der Mehrheit,27295,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
8,No,No,Yes,No,No,31036,20090528,Financement moniste des prestations de soins,2023-09-13T17:16:37Z,None,Antrag der Minderheit IV Aeschi Thomas,Antrag der Mehrheit,27296,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
9,No,No,Yes,No,No,31037,20090528,Financement moniste des prestations de soins,2023-09-13T17:18:13Z,None,Antrag der Minderheit Aeschi Thomas,Antrag der Mehrheit,27297,Loi fédérale sur l’assurance-maladie (LAMal) (Financement uniforme des prestations ambulatoires et des prestations avec hospitalisation),NaN
